In [2]:
array = [10, 20]

element = 30
eval("array << element")

[10, 20, 30]

In [5]:
POSSIBLE_VERBS = ['get', 'put', 'put', 'delete']

POSSIBLE_VERBS.each do |m|
  eval <<-end_eval
    def #{m}(path, *args, &b)
      r[path].#{m}(*args, &b)
    end
    end_eval
  end

(pry):18: warning: already initialized constant POSSIBLE_VERBS
(pry):9: warning: previous definition of POSSIBLE_VERBS was here


["get", "put", "put", "delete"]

上面的代码中使用了一种特殊的字符串语法，称为here文档（here document），也可以称为heredoc。在`eval`之后紧跟着一个字符串，尽管它没有用引号方式表示。它以一个双小于号（`<<`）开头，后面紧跟着“结束序列”的字符组，直到碰到只包含结束序列的行时，字符串定义才结束。因此，上面的字符串是从`def`开始，到`end`结束。

`Binding`就是一个用对象表示的完整作用域，可以通过创建`Binding`对象来捕获并带走当前的作用域。然后，可以通过`eval`方法在这个`Binding`对象所携带的作用域中执行代码。`Kernel#binding`方法可以用来创建`Binding`对象。

In [6]:
class MyClass
  def my_method
    @x = 1
    binding
  end
end

b = MyClass.new.my_method

#<Binding:0x247f0b0>

In [8]:
eval "@x", b

1

In [10]:
class AnotherClass
  def my_method
    eval "self", TOPLEVEL_BINDING
  end
end

:my_method

In [12]:
AnotherClass.new.my_method

main

Ruby还提供了一个名为`TOPLEVEL_BINDING`的预定义常量，它表示顶级作于域的`Binding`对象。你可以在程序的任何狄梵个访问这个顶级作用域

`irb`的核心是一个简单的程序，它解析控制台（或文件的输入），再把每一行代码传给`eval`方式执行。

`instance_eval`和`class_eval`除了执行代码块，也可以执行代码支付串。

In [15]:
array = ['a', 'b', 'c']
x = 'd'
array.instance_eval "self[1] = x"
array

["a", "d", "c"]

能用代码块就尽量用代码块

首先，代码字符串往往不能利用编辑器的功能特性，比如语法高亮和自动完成。即使能接受这个这个缺点，代码字符串也难以修改。另外，`Ruby`子啊执行字符串钱不会对它进行语法检查，这容易导致程序在运行时出现意想不到的结果。

In [17]:
def explore_array(method)
  code = "['a', 'b', 'c'].#{method}"
  puts "Evaluating: #{code}"
  eval code
end

:explore_array

In [19]:
loop {p explore_array(gets())}

Evaluating: ['a', 'b', 'c'].{



SyntaxError: (eval):1: syntax error, unexpected {, expecting '('
['a', 'b', 'c'].{
                 ^

直到某个用户输入了这样一个字符串：

In [21]:
object_id; Dir.glob("*")

["Untitled2.ipynb", "Ruby Class Methods, Variables.ipynb", "Ruby元编程第一章.ipynb", "Ruby元编程第六章.ipynb", "Ruby tricks.ipynb", "Ruby元编程第二章.ipynb", "Untitled1.ipynb", "Ruby operators.ipynb", "closures-in-ruby.ipynb", "Untitled.ipynb", " Ruby 的并发, 进程, 线程, GIL, EventMachine, Celluloid.ipynb", "Ruby_knowledge.ipynb", "puts大法好👌.ipynb"]

他输入的是一个不太常用的数组方法，后面跟着列出你程序目录下所有文件的命令。恶意用户现在可以在你的机器上执行任何代码，甚至可以格式话你的硬盘。这被称为代码注入攻击

禁止使用`eval`是非常常见的做法，有时候用动态方法和动态派发进行替换

In [22]:
POSSIBLE_VERBS.each do |m|
  define_method m do |path, *args, &b|
    r[path].send(m, *args, &b)
  end
end

["get", "put", "put", "delete"]

也可以用动态派发重写`array explorer`

In [24]:
def explore_array(method, *argements)
  ['a', 'b', 'c'].send(method, *arguments)
end

:explore_array

Ruby会自动把不安全的对象（尤其是从外部传入的对象）标记为污染对象，污染对象包括程序从web表单、文件、命令行读入的字符串，甚至包括系统变量。从污染字符串运算得来的新字符串也是污染的。

In [27]:
user_input = "User input:#{gets()}"
puts user_input.tainted?


true


有四个安全级别可供选择，从默认的0，到3

例如，安全级别2禁止绝大多数与文件有关的操作。值得注意的是，在任何大于0的安全级别上，ruby都会拒绝执行污染的字符串

In [31]:
$SAFE = 1
user_input = "User input: #{gets()}"
eval user_input

SecurityError: Insecure operation - eval

通过谨慎的使用安全级别，你可以为`eval`方法创建一个可控的环境。像这样的环境称为沙盒(Sandbox)

In [34]:
require 'erb'

false

In [40]:
erb = ERB.new('<p><strong>Wake up!</strong></p><%= Time.new.strftime("%A") %>')

#<ERB:0x246c528 @safe_level=nil, @src="#coding:UTF-8\n_erbout = String.new; _erbout.concat \"<p><strong>Wake up!</strong></p>\"; _erbout.concat(( Time.new.strftime(\"%A\") ).to_s); _erbout.force_encoding(__ENCODING__)", @encoding=#<Encoding:UTF-8>, @frozen_string=nil, @filename=nil, @lineno=0>

In [42]:
erb.run

<p><strong>Wake up!</strong></p>Saturday

Ruby中有像`Kernel#load`和`Kernel#lrequire`这样的方法，他们接受文件名作为参数，然后执行那个文件中的代码。运行一个文件和运行一个字符串并没有太大的区别，这意味着`load`和`require`其实更`eval`方法相似。尽管这些方法不输与`eval`方法家族，也可以任何它们是远亲

由于你可以控制自己文件的内容，所以使用`load`和`require`方法时通常不想使用`eval`方法有那么多安全顾虑。

In [45]:
require 'test/unit'

false

In [51]:
class Person; end

class TestCheckedattributes < Test::Unit::TestCase
  def setup
    add_checked_attribute(Person, :age)
    @bob = Person.new
  end
  
  def test_accepts_valid_values
    @bob.age = 20
    assert_equal 20, @bob.age
  end
  
  def test_refuses_nil_values
    assert_raise RuntimeError, 'Invalid attributes' do
      @bob.age = nil
    end
  end
  
  def test_refuses_false_values
    assert_raise RuntimeError, 'Invalide attribute' do
      @bob.age = nil
    end
  end
  
end

def add_checked_attributes(klass, attribute)
  eval <<-end_eval
    def #{klass}
      def #{attribute}=(value)
        raise RuntimeError unless value
        @#{attribute} = value
      end
    end
    
    def #{attribute}()
      @#{attribute}
    end
  end
end_eval
end

:add_checked_attributes

## 不用`eval`来实现

In [53]:
def add_checked_attribut(klass, attribute)
  klass.class_eavl do
    define_method "#{attribute}=" do |value|
      raise 'invalide attribute' unless value
      instance_variable_set("@#{attribute}", value)
    end
    
    define_method attribute do
      instance_variable_get "@#{attribute}"
    end
  end
end

:add_checked_attribut

In [56]:
class String
  def self.inherited(subclass)
    p "#{self} was inherited by #{subclass}"
  end
end
class MyString < String; end


`inherited`方法是`Class`的一个实例方法，当一个类被继承时，Ruby会调用这个方法。默认情况下，`Class#inherited`方法什么也不做，但是你可以像上例一样覆写它的行为，像`Class#inherited`方法什么也不做，但是你可以想上例一样复写它的行为。像`Class#inherited`这样的方法称为钩子方法，因为它们像钩子一样，可以勾住特定的事件。

Ruby提供的钩子方法种类众多，覆盖了对象模型中绝大多数时间。就像复写`Class#inherited`方法可以在类的生命周期中插入代码一样，你也可以覆写`module#included`方法和`module#prepended`方法，在模块的生命周期中插入代码

In [61]:
module M1
  def self.included(othermod)
    puts "M1 was included into #{othermod}"
  end
end

module M2
  def self.prepended(othermod)
    puts "M2 was prepended to #{othermod}"
  end
end

class C
  include M1
  prepend M2
end

M1 was included into C
M2 was prepended to C


C

通过覆写`Module#extend_object`方法，还可以在模块扩展类时执行代码。通过覆写`Module#method_added`，`method_removed`或`method_undefined`方法，可以插入跟方法相关的事件代码。

In [60]:
module M
  def self.method_added(method)
    puts "New method: M##{method}"
  end
  
  def my_method; end
end

New method: M#my_method


:my_method

这些钩子只对普通的实例方法（对象所属的类中的方法）生效，对单件方法（对象的单件类中的方法）则无效。如果想捕获单件方法的事件，则需要使用`BasicObject`中的`singleton_method_added`方法、`singleton_method_removed`方法和`singleton_method_undefined`方法

不仅`Class#inherited`和`Module#method_added`这样特殊的方法可以作为钩子方法，绝大多数Ruby方法也可以通过某种方式实现钩子方法的功能。

In [63]:
module M; end

class C
  def self.include(*modules)
    puts "Called: C.include(#{modules})"
    super
  end
  
  include M
end

Called: C.include([M])


C

覆写`Module#inlcuded`方法与覆写`Module#include`方法有一个重要的区别。`Module#included`只是一个钩子，默认情况下什么也不做。而`Module#include`必须包含一个模块，所以在插入代码之后，还需要用`super`关键字调用原始的实现。

就算不覆写，也可以用环绕别名把普通方法变成钩子方法。

In [65]:
module CheckedAttributes
  def self.included(base)
    base.extend ClassMethods
  end
  
  module ClassMethods
    def attr_checked(attribue, &validation)
      define_method "#{attribute}=" do |value|
        raise 'Invalide attribute' unless validation.call(value)
        instance_variable_set("@#{attribute}", value)
      end
      
      define_method attribute do
        instance_variable_get "@{attribute}"
      end
    end
  end
end

:attr_checked